<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=e0b4254f96894387b879dfe8294ee780bc11c76027e903068e99e4039801cdf8
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66


   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-18 09:41:53
-------------------
qualified stocks: 87
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.44 L
Current:  1.43 C
-------------------
Today PnL: 40.19 K (0.28%)
Current PnL: -16.47 L (-11.08%)
CY Booked + Current PnL: -5.19 L (-3.49%)
-------------------
Total profit:  2.23 L
Total loss:  -18.69 L
-------------------
Total Booked + Current PnL: 21.79 L (17.91%)
Total Booked PnL: 38.26 L (31.44%)
Curr Year Booked PnL: 11.28 L (7.89%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 82.09 L (57.42%)
Deployed:  1.22 C
Current:  1.43 C
CAGR/XIRR %: 9.99%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,APOLLOHOSP,8285.00,-13.75,56.0,H-LC,10.85,172447.0,16270.0,9829.0,-0.63,10.42,5.70,16.71,22.0,1.66,1.24,29.67,X40N,BTT,HEALTHCARE
77,TTKPRESTIG,770.00,105.55,60.0,M-SC,3.96,88549.0,-12228.0,12317.0,-0.12,-12.13,13.91,0.09,245.0,-0.99,0.64,16.33,OX40N,NTT,DURABLES
57,RELIANCE,1533.00,-11.79,61.0,H-LC,5.19,221239.0,10873.0,17898.0,0.31,5.17,8.09,13.68,37.0,0.61,1.60,22.55,XY25,NTT,REFINERIES
50,NATIONALUM,247.44,-43.66,63.0,H-MC,6.80,112620.0,12116.0,18526.0,-0.36,12.06,16.45,30.49,79.0,0.65,0.81,48.37,MH,ATH,METALS
42,ITC,452.00,-37.50,51.0,H-LC,2.52,201248.0,1110.0,19783.0,0.55,0.55,9.83,10.44,4.0,0.06,1.45,6.15,X40,NTT,FMCG


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,43.54,62.0,M-MC,6.95,227046.0,2084.0,165562.0,-0.68,0.93,72.92,74.53,192.0,0.01,1.64,33.48,XY24,BTT,STEEL
35,ICICIPRULI,790.00,-20.88,44.0,H-MC,2.24,136097.0,309.0,42449.0,0.35,0.23,31.19,31.48,107.0,0.01,0.98,12.37,X40,ATH,INSURANCE
30,HAVELLS,2069.16,-0.09,57.0,H-MC,1.86,247712.0,-19.0,75082.0,-0.35,-0.01,30.31,30.30,92.0,-0.00,1.79,13.28,X40,ATH,ELECTRICAL
52,PGHH,17973.08,-30.68,54.0,H-MC,4.20,200835.0,15.0,68766.0,0.04,0.01,34.24,34.25,80.0,0.00,1.45,5.21,X40,ATH,FMCG


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-17.18,39.0,H-LC,6.39,298488.0,-17354.0,130021.0,-0.68,-5.49,43.56,35.67,5.0,-0.13,2.15,7.60,X40N,ATH,FMCG
6,ASIANPAINT,3465.66,-12.93,48.0,H-LC,7.84,219850.0,-31918.0,85126.0,0.19,-12.68,38.72,21.13,27.0,-0.37,1.59,18.26,X40,ATH,PAINTS
42,ITC,452.00,-37.50,51.0,H-LC,2.52,201248.0,1110.0,19783.0,0.55,0.55,9.83,10.44,4.0,0.06,1.45,6.15,X40,NTT,FMCG
44,JIOFIN,387.00,0.15,51.0,H-LC,13.22,272061.0,4696.0,60751.0,0.16,1.76,22.33,24.48,48.0,0.08,1.96,57.69,XY24,BTT,FINANCE
20,CIPLA,1795.00,-19.78,52.0,H-LC,5.76,214158.0,9658.0,31760.0,0.23,4.72,14.83,20.25,10.0,0.30,1.55,13.87,X40N,BTT,PHARMA


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,1.16,72.0,H-LC,1.31,251212.0,5061.0,74158.0,2.94,2.06,29.52,32.18,16.0,0.07,1.81,38.84,X200,ATH,IT
42,ITC,452.0,-37.50,51.0,H-LC,2.52,201248.0,1110.0,19783.0,0.55,0.55,9.83,10.44,4.0,0.06,1.45,6.15,X40,NTT,FMCG
51,NESTLEIND,1377.0,-8.60,56.0,H-LC,4.38,281268.0,15842.0,40953.0,-0.19,5.97,14.56,21.40,11.0,0.39,2.03,14.22,XY25,NTT,FMCG
57,RELIANCE,1533.0,-11.79,61.0,H-LC,5.19,221239.0,10873.0,17898.0,0.31,5.17,8.09,13.68,37.0,0.61,1.60,22.55,XY25,NTT,REFINERIES
20,CIPLA,1795.0,-19.78,52.0,H-LC,5.76,214158.0,9658.0,31760.0,0.23,4.72,14.83,20.25,10.0,0.30,1.55,13.87,X40N,BTT,PHARMA


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,ATULAUTO,844.0,3997.30,66.0,M-SC,5.77,127759.0,-19609.0,71430.0,3.38,-13.31,55.91,35.16,236.0,-0.27,0.92,30.49,XY24,NTT,AUTO
48,LTIM,7230.2,1.16,72.0,H-LC,1.31,251212.0,5061.0,74158.0,2.94,2.06,29.52,32.18,16.0,0.07,1.81,38.84,X200,ATH,IT
40,INFY,2275.0,-15.29,60.0,H-LC,8.39,330299.0,17411.0,154283.0,1.86,5.56,46.71,54.87,3.0,0.11,2.38,12.58,X40,BTT,IT
86,WIPRO,420.0,-8.80,62.0,M-LC,5.61,160351.0,9406.0,100043.0,1.76,6.23,62.39,72.51,53.0,0.09,1.16,12.63,XR,NTT,IT
11,BANDHANBNK,400.0,-10.58,52.0,H-SC,5.12,223592.0,-54967.0,312403.0,1.57,-19.73,139.72,92.42,151.0,-0.18,1.61,29.22,XY24,NTT,BANKS


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,HINDZINC,730.22,20.84,58.0,M-LC,1.35,196946.0,-8130.0,120708.0,-1.46,-3.96,61.29,54.89,52.0,-0.07,1.42,19.18,X5K,ATH,METALS
27,GREENPANEL,537.00,234.63,63.0,M-SC,2.42,149352.0,-33726.0,112701.0,-1.26,-18.42,75.46,43.14,230.0,-0.30,1.08,40.49,XY24,NTT,MISC
22,COFFEEDAY,80.00,-43.66,50.0,L-SC,42.25,84096.0,-29453.0,69505.0,-1.20,-25.94,82.65,35.27,268.0,-0.42,0.61,105.83,XR,NTT,HOTELS
55,RAJOOENG,143.33,-35.87,39.0,H-SC,11.56,97220.0,-5280.0,46111.0,-0.80,-5.15,47.43,39.83,114.0,-0.11,0.70,1.62,AR,ATH,MISC
70,SYMPHONY,1306.00,-41.90,41.0,M-SC,18.29,123992.0,-47149.0,47092.0,-0.71,-27.55,37.98,-0.03,196.0,-1.00,0.90,1.64,OX40N,NTT,DURABLES


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.0,105.55,60.0,M-SC,3.96,88549.0,-12228.0,12317.0,-0.12,-12.13,13.91,0.09,245.0,-0.99,0.64,16.33,OX40N,NTT,DURABLES
78,UJJIVANSFB,60.0,96.03,55.0,H-SC,9.53,123606.0,-18873.0,38392.0,0.07,-13.25,31.06,13.70,163.0,-0.49,0.89,45.89,OX40N,NTT,BANKS
19,CERA,9475.0,-19.33,51.0,H-SC,2.08,147177.0,-28726.0,70748.0,0.39,-16.33,48.07,23.89,149.0,-0.41,1.06,27.11,OX40N,NTT,CERAMICS
45,KANSAINER,340.0,-67.42,61.0,H-SC,1.45,224820.0,-44847.0,81183.0,0.12,-16.63,36.11,13.47,138.0,-0.55,1.62,13.71,XY24,NTT,PAINTS
72,TATAELXSI,9161.0,-19.17,55.0,H-MC,7.64,109250.0,-22585.0,64807.0,1.18,-17.13,59.32,32.03,98.0,-0.35,0.79,22.37,OX40N,NTT,IT


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-24.83,62.0,H-LC,12.33,300048.0,-45908.0,120769.0,0.61,-13.27,40.25,21.64,1.0,-0.38,2.17,6.29,X40,ATH,IT
40,INFY,2275.00,-15.29,60.0,H-LC,8.39,330299.0,17411.0,154283.0,1.86,5.56,46.71,54.87,3.0,0.11,2.38,12.58,X40,BTT,IT
42,ITC,452.00,-37.50,51.0,H-LC,2.52,201248.0,1110.0,19783.0,0.55,0.55,9.83,10.44,4.0,0.06,1.45,6.15,X40,NTT,FMCG
83,VBL,671.64,-17.18,39.0,H-LC,6.39,298488.0,-17354.0,130021.0,-0.68,-5.49,43.56,35.67,5.0,-0.13,2.15,7.60,X40N,ATH,FMCG
1,ABB,7934.00,-37.73,67.0,H-LC,10.35,260275.0,-1344.0,120559.0,0.68,-0.51,46.32,45.57,7.0,-0.01,1.88,10.75,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,43.54,62.0,M-MC,6.95,227046.0,2084.0,165562.0,-0.68,0.93,72.92,74.53,192.0,0.01,1.64,33.48,XY24,BTT,STEEL
15,BLUESTARCO,2326.38,13.46,64.0,H-MC,3.56,195820.0,31150.0,36814.0,0.93,18.92,18.80,41.28,89.0,0.85,1.41,28.41,X40N,ATH,AC
84,VOLTAS,1918.49,0.60,56.0,H-MC,2.47,211035.0,19293.0,76732.0,-0.35,10.06,36.36,50.08,99.0,0.25,1.52,17.54,XY25,ATH,AC
18,CAMS,5211.76,-0.29,65.0,H-SC,3.84,113397.0,11393.0,32534.0,0.90,11.17,28.69,43.06,122.0,0.35,0.82,30.20,X40N,ATH,MISC
24,DABUR,735.00,-3.84,54.0,H-MC,6.74,209332.0,12304.0,77327.0,0.30,6.24,36.94,45.49,102.0,0.16,1.51,20.10,XY24,BTT,FMCG


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,247.44,-43.66,63.0,H-MC,6.80,112620.0,12116.0,18526.0,-0.36,12.06,16.45,30.49,79.0,0.65,0.81,48.37,MH,ATH,METALS
85,WHIRLPOOL,2270.00,-40.41,52.0,M-SC,0.49,100658.0,9160.0,69595.0,0.04,10.01,69.14,86.07,223.0,0.13,0.73,46.17,XR,NTT,DURABLES
37,INDIAMART,4850.92,-51.84,43.0,H-SC,11.68,134562.0,11226.0,122532.0,-0.47,9.10,91.06,108.45,119.0,0.09,0.97,33.49,AR,ATH,MISC
86,WIPRO,420.00,-8.80,62.0,M-LC,5.61,160351.0,9406.0,100043.0,1.76,6.23,62.39,72.51,53.0,0.09,1.16,12.63,XR,NTT,IT
12,BANKINDIA,190.00,-29.19,66.0,H-MC,6.13,183945.0,4137.0,108270.0,-0.03,2.30,58.86,62.52,88.0,0.04,1.33,37.17,XR,NTT,BANKS


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RAJESHEXPO,518.00,1896.21,67.0,L-SC,3.05,54455.0,-82722.0,82815.0,0.32,-60.30,152.08,0.07,267.0,-1.00,0.39,35.43,OX40N,NTT,JEWELLERY
7,ASIANTILES,137.00,6727.78,53.0,L-SC,5.19,74889.0,-18921.0,95536.0,1.07,-20.17,127.57,81.67,269.0,-0.20,0.54,45.03,XR,NTT,CERAMICS
22,COFFEEDAY,80.00,-43.66,50.0,L-SC,42.25,84096.0,-29453.0,69505.0,-1.20,-25.94,82.65,35.27,268.0,-0.42,0.61,105.83,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-3.83,49.0,H-SC,12.75,89548.0,-11415.0,105944.0,-0.54,-11.31,118.31,93.63,148.0,-0.11,0.65,33.38,SR,ATH,CHEMICALS
49,MASFIN,398.61,-16.89,53.0,H-SC,12.99,94635.0,-3345.0,24946.0,-0.28,-3.41,26.36,22.05,152.0,-0.13,0.68,37.56,XR,ATH,FINANCE


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,ATULAUTO,844.0,3997.30,66.0,M-SC,5.77,127759.0,-19609.0,71430.0,3.38,-13.31,55.91,35.16,236.0,-0.27,0.92,30.49,XY24,NTT,AUTO
75,TITAGARH,1548.0,1.62,69.0,H-SC,3.25,166469.0,-25546.0,104426.0,0.96,-13.30,62.73,41.08,158.0,-0.24,1.20,41.06,XY24,NTT,ENGINEERING
54,RAJESHEXPO,518.0,1896.21,67.0,L-SC,3.05,54455.0,-82722.0,82815.0,0.32,-60.30,152.08,0.07,267.0,-1.00,0.39,35.43,OX40N,NTT,JEWELLERY
22,COFFEEDAY,80.0,-43.66,50.0,L-SC,42.25,84096.0,-29453.0,69505.0,-1.20,-25.94,82.65,35.27,268.0,-0.42,0.61,105.83,XR,NTT,HOTELS
61,SAMMAANCAP,326.0,-170.45,61.0,M-SC,2.61,81768.0,-20442.0,113829.0,0.90,-20.00,139.21,91.37,208.0,-0.18,0.59,32.74,XY25,NTT,FINANCE


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.86
1,25,43.33
2,50,73.74


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.99
LC    33.01
MC    22.94
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.92
X40      15.26
X40N     13.00
XR       11.63
AR        8.65
OX40N     8.14
XY25      8.13
X200      1.81
MH        1.69
X5K       1.42
SR        1.29
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    28.96
H-LC    26.35
H-MC    20.02
M-SC    13.49
M-LC     5.59
M-MC     2.62
L-SC     1.54
L-LC     1.07
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.57,-11.75,72.68
FMCG,11.71,-4.25,40.73
FINANCE,9.62,-15.32,60.48
MISC,7.41,-12.05,74.36
BANKS,6.23,-15.44,76.17
PAINTS,5.80,-14.88,32.00
ELECTRICAL,5.68,-4.69,62.39
AC,3.64,4.24,28.18
INSURANCE,3.39,-7.13,43.87


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2834609.0,22
XR,1299287.0,14
AR,1297752.0,9
X40,734992.0,10
X40N,609235.0,10
OX40N,518779.0,10
XY25,395997.0,6
SR,242943.0,2
X5K,120708.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3397267.0,29
M-SC,1376098.0,17
H-LC,1223095.0,15
H-MC,1177796.0,15
M-LC,375022.0,4
M-MC,315030.0,2
L-SC,247856.0,3
L-MC,60519.0,1
L-LC,36281.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1021955.0      6
M-SC       XY24       795844.0      7
H-SC       AR         783994.0      5
           XR         751485.0      7
H-MC       XY24       486758.0      4
H-LC       X40        450788.0      5
           AR         353059.0      2
M-MC       XY24       315030.0      2
H-SC       X40N       286260.0      4
           OX40N      248652.0      4
           SR         242943.0      2
H-LC       X40N       225488.0      4
H-MC       X40        214912.0      4
           XY25       187036.0      2
L-SC       XR         165041.0      2
M-SC       AR         160699.0      2
M-LC       XY24       154271.0      2
M-SC       OX40N      122505.0      4
M-LC       X5K        120708.0      1
M-SC       XR         113929.0      2
           XY25       113829.0      1
H-MC       XR         108270.0      1
M-LC       XR         100043.0      1
H-MC       X40N        97487.0      2
L-SC       OX40N       82815.0      1
H-LC       X200        74158.0      1
M-SC       X40         69292.0      1
H-MC       OX40N       64807.0      1
H-SC       MH          61978.0      1
H-LC       XY24        60751.0      1
L-MC       XR          60519.0      1
H-LC       XY25        58851.0      2
L-LC       XY25        36281.0      1
H-MC       MH          18526.0      1

In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 24.0 seconds
